In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Path to your CSV file
file_path = '/content/drive/MyDrive/Pembelajaran Mesin/tugasxgboost2.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
df.head()

,"fixed acidity;""volatile acidity"";""citric acid"";""residual sugar"";""chlorides"";""free sulfur dioxide"";""total sulfur dioxide"";""density"";""pH"";""sulphates"";""alcohol"";""quality"""
0,7;0.27;0.36;20.7;0.045;45;170;1.001;3;0.45;8.8;6
1,6.3;0.3;0.34;1.6;0.049;14;132;0.994;3.3;0.49;9...
2,8.1;0.28;0.4;6.9;0.05;30;97;0.9951;3.26;0.44;1...
3,7.2;0.23;0.32;8.5;0.058;47;186;0.9956;3.19;0.4...
4,7.2;0.23;0.32;8.5;0.058;47;186;0.9956;3.19;0.4...


In [ ]:
print(df.columns.tolist())

['fixed acidity;"volatile acidity";"citric acid";"residual sugar";"chlorides";"free sulfur dioxide";"total sulfur dioxide";"density";"ph";"sulphates";"alcohol";"quality"']


In [ ]:
# Select specific predictors
predictors = df

# Set the target variable
target = df


In [ ]:
from sklearn.model_selection import train_test_split

# Memisahkan data menjadi data latih (training) dan data validasi (validation)
# Using 'predictors' for X and 'target' for y
X_train, X_valid, y_train, y_valid = train_test_split(predictors, target, test_size=0.2, random_state=0)

# test_size=0.2 menentukan bahwa 20% data akan digunakan sebagai data validasi, sementara 80% sisanya untuk data latih
# random_state=0 memastikan hasil split data konsisten jika kode dijalankan kembali

In [ ]:
from xgboost import XGBRegressor

# Display the parameters of the XGBRegressor
xgbr = XGBRegressor()
print(xgbr.get_params())


{'objective': 'reg:squarederror', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': None, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': None, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': None, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': None, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}


In [ ]:
import numpy as np

# Check for NaN or infinite values in y_train and y_valid
print("NaN in y_train:", np.isnan(y_train).sum())
print("NaN in y_valid:", np.isnan(y_valid).sum())
print("Infinite in y_train:", np.isinf(y_train).sum())
print("Infinite in y_valid:", np.isinf(y_valid).sum())


NaN in y_train: fixed_acidity;"volatile_acidity";"citric_acid";"residual_sugar";"chlorides";"free_sulfur_dioxide";"total_sulfur_dioxide";"density";"ph";"sulphates";"alcohol";"quality"    3918
dtype: int64
NaN in y_valid: fixed_acidity;"volatile_acidity";"citric_acid";"residual_sugar";"chlorides";"free_sulfur_dioxide";"total_sulfur_dioxide";"density";"ph";"sulphates";"alcohol";"quality"    980
dtype: int64
Infinite in y_train: fixed_acidity;"volatile_acidity";"citric_acid";"residual_sugar";"chlorides";"free_sulfur_dioxide";"total_sulfur_dioxide";"density";"ph";"sulphates";"alcohol";"quality"    0
dtype: int64
Infinite in y_valid: fixed_acidity;"volatile_acidity";"citric_acid";"residual_sugar";"chlorides";"free_sulfur_dioxide";"total_sulfur_dioxide";"density";"ph";"sulphates";"alcohol";"quality"    0
dtype: int64


In [ ]:
# Remove rows with NaN or infinite values
X_train = X_train[~np.isnan(y_train) & ~np.isinf(y_train)]
y_train = y_train[~np.isnan(y_train) & ~np.isinf(y_train)]
X_valid = X_valid[~np.isnan(y_valid) & ~np.isinf(y_valid)]
y_valid = y_valid[~np.isnan(y_valid) & ~np.isinf(y_valid)]


In [ ]:
# Replace NaN values with mean of y_train (or another strategy of choice)
y_train = np.nan_to_num(y_train, nan=np.nanmean(y_train), posinf=np.nanmean(y_train), neginf=np.nanmean(y_train))
y_valid = np.nan_to_num(y_valid, nan=np.nanmean(y_valid), posinf=np.nanmean(y_valid), neginf=np.nanmean(y_valid))


<ipython-input-60-30d4d3783792>:2: RuntimeWarning: Mean of empty slice
  y_train = np.nan_to_num(y_train, nan=np.nanmean(y_train), posinf=np.nanmean(y_train), neginf=np.nanmean(y_train))
<ipython-input-60-30d4d3783792>:3: RuntimeWarning: Mean of empty slice
  y_valid = np.nan_to_num(y_valid, nan=np.nanmean(y_valid), posinf=np.nanmean(y_valid), neginf=np.nanmean(y_valid))


In [ ]:
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

# Mengecek dan membersihkan nilai NaN, infinity, atau nilai yang terlalu besar pada y_train dan y_valid
def clean_target(y):
    # Menghapus nilai NaN, infinity, atau nilai yang terlalu besar pada y
    y_cleaned = np.where(np.isnan(y), np.nanmean(y), y)  # Ganti NaN dengan rata-rata
    y_cleaned = np.where(np.isinf(y_cleaned), np.nanmean(y), y_cleaned)  # Ganti infinity dengan rata-rata
    y_cleaned = np.where(y_cleaned > 1e10, np.nanmean(y), y_cleaned)  # Atasi nilai yang terlalu besar
    return y_cleaned

# Bersihkan y_train dan y_valid
y_train = clean_target(y_train)
y_valid = clean_target(y_valid)

# Konfirmasi ulang bahwa tidak ada nilai NaN atau infinity
print("Jumlah NaN di y_train setelah pembersihan:", np.isnan(y_train).sum())
print("Jumlah NaN di y_valid setelah pembersihan:", np.isnan(y_valid).sum())
print("Jumlah infinity di y_train setelah pembersihan:", np.isinf(y_train).sum())
print("Jumlah infinity di y_valid setelah pembersihan:", np.isinf(y_valid).sum())

# Inisialisasi model XGBoost
model = XGBRegressor(random_state=0)

# Melatih model dengan data latih
model.fit(X_train, y_train)

# Membuat prediksi pada data validasi
predictions = model.predict(X_valid)

# Menghitung Mean Absolute Error (MAE) pada data validasi
mae = mean_absolute_error(y_valid, predictions)
print("Mean Absolute Error pada data validasi:", mae)
